In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
import time
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np

# Academic Ranking of World Universities
# Shanghai Ranking

## Web scraping dos dados do ranking Shanghai

Para o web scraing dos dados será usada a biblioteca selenium

In [7]:
#Necessário caso ChromeDriver não esteja no path
'''
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
'''

'\nfrom selenium import webdriver\nfrom selenium.webdriver.chrome.service import Service\nfrom webdriver_manager.chrome import ChromeDriverManager\n\ndriver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))\n'

### Scrap de rankings individuais

In [2]:
ano='2008' #especificar o ano do ranking 
website='https://www.shanghairanking.com/rankings/arwu/'+ano
driver = webdriver.Chrome()
driver.get(website) #abre a pagina
driver.maximize_window()  #maximizar a página

Na primeira coluna da tabela estão os seguintes headers:

- World Rank;
- Institution;
- Country/Region;
- National/Regional Rank;
- Total Score;
- Menu com as feutures:
    - Alumni;
    - Award;
    - HiCi;
    - N&S;
    - PUB;
    - PCP.

As features estão em um menu dropdown. Portanto durante o processo de scraping deverá ocorrer interação com esse menu para que ser possível coletar os dados para cada uma delas. Além disso, os dados para todas as universidades estão 
em diversas páginas, sendo assim será necessário interagir com a numeração das páginas para coletar os dados do completos do ranking


In [3]:
features=["Alumni",'Award','HiCi','N&S','PUB','PCP'] #features calculadas para cada universidade. O significado de cada uma dela pode ser encontrado no notebook com a análise dos dados dos rankings
#Essas features estão em um menu dropdown na página

#Dicionário que receberá os dados
ranking={'World Rank' :[],
              'Institution':[],
              'Country'    :[],
              'Regional Rank':[],
              'Total Score':[],
                'Alumni': [],
                'Award' : [],
                'HiCi'  : [],
                'N&S'  : [],
                'PUB'   : [],
                'PCP'   : []}


 

In [ ]:
#Path onde serão salvos os dados
path_to_save='/home/rodolfo/Insync/rodolfopcruz2@gmail.com/Google Drive/Estudo/Python_Projects/education_data/dados/Shanghai World University Rankings/web_scraping/'

In [4]:
#número de páginas
#os ranking estão divididos em diversas páginas
#em geral em cada página estão os dados para 30 universidades
pagination=driver.find_element('xpath',"//ul[contains(@class,'pagination')]")
paginas=pagination.find_elements('xpath','./li') #botões com as numerações das páginas
num_paginas=paginas[-2].text #o promeiro botão é um seta para recuar e o último uma seta para avançar. Entre esses dois estão os botões com os números das páginas
num_paginas #total de pádinas onde estão contidos os dados

'17'

In [5]:
#web sraping dos dados do ranking do ano selecionado

botao_next=pagination.find_element('xpath','//li[@class=" ant-pagination-next"]') #Botaão para avançar a página
time.sleep(3)

pagina_atual=1 #primeira página
while pagina_atual<=int(num_paginas): #varrer todas as páginas com os dados do ranking

    botoes_header=driver.find_elements(by='xpath',value='//th//img') #localizar botão dropdown. Ele está nos headers da tabela. Ao clicar nesse botão o menu dropdow é aberto
    botoes_header=botoes_header[-1] #nos headers estão dois botões dropdown. O primeiro seleciona o país e o segundo a feature. O botão desejado é o segundo.
        
    opcoes=driver.find_elements(by='xpath',value='//th') #localizar as opções contidas nos menus dropdowns no header da tabela
    opcoes=opcoes[-1] #segundo menu dropdown no header da tabela
    opcoes=opcoes.find_elements('xpath','.//li') #pegar as opções contidas no menu dropdown. Opcoes é uma lista em que cada elemento corresponde a uma feature
        
    table=driver.find_element(by='xpath',value="//tbody") #dados na tabela sem o header (somente a tabela com os dados, sem os nomes das colunas)
    table_rows=table.find_elements(by='xpath',value="./tr") #rows da tabela

    first_feature=True #indicar se é a primeira feature do meno dropdown.  tabela na página tem 6 colunas. Na sexta está um menui dropdrown para seleção da feature desejada.
                       #Alterando a feature no menu dropdown nao tem nenhum efeito nas 5 colunas anteriores. Então se first_feature=True será feito o scrap dos dados das seis colunas.Caso 
                       #contrário somente da última coluna, uma vez que os dados das demais colunas já foram coletados
    
    for feature in features:  #selecionar uma feature     
        botoes_header.click() #clicar no botão para abrir o menu dropdown
        time.sleep(1)
        for opcao in opcoes: #cada opcao é uma das features no menu dropdown
            if opcao.text==feature: #opcao.text retorna o nome da feature. Seleciona a opção desejada da feature no menu dropdown
                opcao.click() #clica na opcao
                if first_feature: #se true fará scrap de seis colunas
                     for row in table_rows: #loop por todas as linhas da tabela
                        td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                        ranking['World Rank'].append(td[0].text)#primeira coluna é a posicao no ranking mundial
                        ranking['Institution'].append(td[1].text)#primeira coluna é a posicao no ranking mundial
                        codigo_pais=td[2].find_element(by='xpath',value='div[@class="region-img"]') #na coluna com o pais está uma url para a bandeira do pais
                        ranking['Country'].append(codigo_pais.get_attribute("style").split('/')[-1].split('.')[0]) #extrair o nome do pais da url
                        ranking['Regional Rank'].append(td[3].text) #terceria coluna é a posição da universidade no ranking do próprio país
                        ranking['Total Score'].append(td[4].text)   #pontuação final obtida pela universidade
                        ranking[feature].append(td[5].text)         #primeira das 6 fatures no menu dropdown
                        first_feature=False #first_feature=False para que na próxima iteração sejam coletados somente os dados da última coluna                    
                else:
                    for row in table_rows: #loop por todas as linhas da tabela
                        td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                        ranking[feature].append(td[5].text) #coletar os dados das features no menu dropdown                
                time.sleep(5)
                break
    print(pagina_atual)
    time.sleep(3)  
    botao_next.click() #clicar no botão para avançar a página
    #print('')
    time.sleep(3)
    botao_top=driver.find_element('xpath','//div[@class="back-top"]') #botão para voltar ao topo da página
    botao_top.click() #clicar no botão para voltar ao topo da página
    time.sleep(10)
    pagina_atual+=1
ranking=ranking=pd.DataFrame.from_dict(ranking) #converter o dicionário para dataframe
ranking.to_csv(path_to_save+'ranking_'+ano+'.csv',index=False) #salvar o dataframe no diretório especificado. 
driver.quit() #fechar a página
              


30
30
30
30
30
30
30

1
30
30
30
30
30
30
30

2
30
30
30
30
30
30
30

3
30
30
30
30
30
30
30

4
30
30
30
30
30
30
30

5
30
30
30
30
30
30
30

6
30
30
30
30
30
30
30

7
30
30
30
30
30
30
30

8
30
30
30
30
30
30
30

9
30
30
30
30
30
30
30

10
30
30
30
30
30
30
30

11
30
30
30
30
30
30
30

12
30
30
30
30
30
30
30

13
30
30
30
30
30
30
30

14
30
30
30
30
30
30
30

15
30
30
30
30
30
30
30

16
23
23
23
23
23
23
23

17


In [6]:
for key in ranking.keys():
    print(key)
    print(len(ranking[key]))

World Rank
503
Institution
503
Country
503
Regional Rank
503
Total Score
503
Alumni
503
Award
503
HiCi
503
N&S
503
PUB
503
PCP
503


In [22]:
'''
table=driver.find_element(by='xpath',value="//tbody") #dados na tabela sem os header
table_rows=table.find_elements(by='xpath',value="./tr") #rows da tabela

for row in table_rows: #loop por todas as linhas da tabela
                        td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                        ranking['World Rank'].append(td[0].text)#primeira coluna é a posicao no ranking mundial
                        print(td[0].text)
                        print(td[1].text)
                        print(td[2].text)
                        print(td[3].text)
                        print(td[4].text)
                        print(td[5].text)
                        print('')

                        ranking['Institution'].append(td[1].text)#primeira coluna é a posicao no ranking mundial
                        codigo_pais=td[2].find_element(by='xpath',value='div[@class="region-img"]')
                        ranking['Country'].append(codigo_pais.get_attribute("style").split('/')[-1].split('.')[0])
                        ranking['Regional Rank'].append(td[3].text)
                        ranking['Total Score'].append(td[4].text)
                        ranking["Alumni"].append(td[5].text)
'''

151-200
Cardiff University

16-22

0.0

151-200
Colorado State University

70-87

0.0

151-200
Delft University of Technology

5-7

14.8

151-200
Florida State University

70-87

0.0

151-200
Iowa State University

70-87

0.0

151-200
Joseph Fourier University (Grenoble 1)

6

0.0

151-200
National Autonomous University of Mexico

1

16.0

151-200
National Taiwan University

1

14.8

151-200
Newcastle University

16-22

0.0

151-200
Oregon Health and Science University

70-87

0.0

151-200
Queen's University

6-8

0.0

151-200
Seoul National University

1

0.0

151-200
State University of New York at Stony Brook

70-87

0.0

151-200
Technical University of Denmark

3

8.6

151-200
The University of Alabama at Birmingham

70-87

0.0

151-200
The University of Calgary

6-8

0.0

151-200
The University of Connecticut - Storrs

70-87

14.8

151-200
The University of Hong Kong

1

0.0

151-200
The University of Texas Health Science Center at Houston

70-87

0.0

151-200
The University of Te

In [40]:
x=pd.read_csv(path_to_save+'ranking_2004.csv')
x.tail()

,World Rank,Institution,Country,Regional Rank,Total Score,Alumni,Award,HiCi,N&S,PUB,PCP
497,404-502,University of Tromso,no,4,NaN,0.0,0.0,8.7,3.0,23.3,7.8
498,404-502,University of Wuppertal,de,38-43,NaN,0.0,0.0,12.4,6.9,18.1,8.3
499,404-502,University of Zaragoza,es,5-9,NaN,0.0,0.0,8.7,2.5,27.6,8.7
500,404-502,West Virginia University,us,140-170,NaN,0.0,0.0,0.0,6.4,28.6,8.7
501,404-502,York University,ca,20-23,NaN,0.0,0.0,0.0,12.8,25.5,8.5


# 2003 a 2023

In [64]:
ano=[str(ano) for ano in np.arange(2006,2024)]
#website='https://www.shanghairanking.com/rankings/arwu/2023'
#driver = webdriver.Chrome()
#driver.get(website) #abre a pagina
#driver.maximize_window()  #maximizar a página

In [12]:
anos=[str(i) for i in range(2006,2024)]
for ano in anos:
    website='https://www.shanghairanking.com/rankings/arwu/'+ano #abrir a pagina do ano selecionado
    driver = webdriver.Chrome()
    driver.get(website) #abre a pagina
    driver.maximize_window()  #maximizar a página 
    time.sleep(20) #aguardar o carregamento da página


    #Dicionário que receberá os dados
    ranking={'World Rank' :[],
              'Institution':[],
              'Country'    :[],
              'Regional Rank':[],
              'Total Score':[],
                'Alumni': [],
                'Award' : [],
                'HiCi'  : [],
                'N&S'  : [],
                'PUB'   : [],
                'PCP'   : []}
    
    #verificar o número de páginas
    pagination=driver.find_element('xpath',"//ul[contains(@class,'pagination')]")
    paginas=pagination.find_elements('xpath','./li')
    num_paginas=paginas[-2].text
    num_paginas

    botao_next=pagination.find_element('xpath','//li[@class=" ant-pagination-next"]') #botão para avançar a página
    time.sleep(3)

    pagina_atual=1 #primera página do ranking
    while pagina_atual<=int(num_paginas): #varrer todas as páginas do ranking
        
        botoes_header=driver.find_elements(by='xpath',value='//th//img') #localizar botão dropdown para seleção das features. Ele está nos headers da tabela. 
                                                                        #Ao clicar nesse botão o menu dropdow é aberto
        botoes_header=botoes_header[-1] #nos headers estão dois botões dropdown. O primeiro seleciona o país e o segundo a feature. O botão desejado é o segundo.
                                        #botões_header é o botão qie deve ser clicado para abertura do menu dropdown
        opcoes=driver.find_elements(by='xpath',value='//th') #menus dropdown no header da tabela. opcoes são as opções contidas no menu dropdown
        opcoes=opcoes[-1] #segundo menu dropdown no header da tabela
        opcoes=opcoes.find_elements('xpath','.//li') #pegar as opções contidas no menu dropdown. Opcoes é uma lista em que cada elemento corresponde a uma feature
        
        table=driver.find_element(by='xpath',value="//tbody") #dados na tabela sem os headers
        table_rows=table.find_elements(by='xpath',value="./tr") #rows da tabela

        '''
        first_feature é usada para determinar quais colunas da tabela serão coletadas. A tabela tem várias colunas, somente em uma delas a feature na coluna pode ser selecionada.
        Quando a feature na coluna é selecionada, somente os dados da coluna são alterados. Então quando first_feature==true os dados de todas as colunas serão coletados, caso contrário
        serão coletados somentes os dados da coluna com a feature selecionada
        '''
        first_feature=True 
        for feature in features:       
          botoes_header.click() #clicar no botão para abrir o menu dropdown
          time.sleep(5)
          for opcao in opcoes: #cada opcao é uma das features no menu dropdown
            if opcao.text==feature: #opcao.text retorna o nome da feature. Seleciona a opção desejada da feature
              opcao.click()
              if first_feature: #coletar dados de todas as colunas na tabela
                for row in table_rows: #loop por todas as linhas da tabela
                  td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                  ranking['World Rank'].append(td[0].text)#primeira coluna é a posicao no ranking mundial
                  ranking['Institution'].append(td[1].text)#primeira coluna é a posicao no ranking mundial
                  codigo_pais=td[2].find_element(by='xpath',value='div[@class="region-img"]') #no pais está url de uma imagem da bandeira do pais
                  ranking['Country'].append(codigo_pais.get_attribute("style").split('/')[-1].split('.')[0]) #extrair o nome do pais daurl
                  ranking['Regional Rank'].append(td[3].text)
                  ranking['Total Score'].append(td[4].text)
                  ranking[feature].append(td[5].text)
                  first_feature=False #na próxima interação somente serão coletados os dados da feature selecionada no menu dropdown. As demais colunas da tabela já foram coletados
              else:
                for row in table_rows: #loop por todas as linhas da tabela
                  td=row.find_elements(by='xpath',value='./td') #td são as colunas da tabela.
                  ranking[feature].append(td[5].text)  
              time.sleep(5)
              break
        time.sleep(2)
        botao_next.click() #avançar pagina
        time.sleep(2)
        botao_top=driver.find_element('xpath','//div[@class="back-top"]') #botão para voltar ao topo da página
        botao_top.click() #clicar no botão
        time.sleep(5)     
        print('Página atual: {}'.format(pagina_atual))
        print('Ano: {}'.format(ano))
        print('')
        pagina_atual+=1
    ranking=pd.DataFrame.from_dict(ranking)
    ranking.to_csv(path_to_save+'ranking_'+ano+'.csv',index=False)
    driver.quit()  
        
        
    



 

Página atual: 1
Ano: 2006

Página atual: 2
Ano: 2006

Página atual: 3
Ano: 2006

Página atual: 4
Ano: 2006

Página atual: 5
Ano: 2006

Página atual: 6
Ano: 2006

Página atual: 7
Ano: 2006



ElementClickInterceptedException: Message: element click intercepted: Element <img data-v-24ba571f="" src="data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz4KPCEtLSBHZW5lcmF0b3I6IEFkb2JlIElsbHVzdHJhdG9yIDI0LjAuMSwgU1ZHIEV4cG9ydCBQbHVnLUluIC4gU1ZHIFZlcnNpb246IDYuMDAgQnVpbGQgMCkgIC0tPgo8c3ZnIHZlcnNpb249IjEuMSIgaWQ9IuWbvuWxgl8xIiB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHhtbG5zOnhsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIiB4PSIwcHgiIHk9IjBweCIKCSB2aWV3Qm94PSIwIDAgMTQgMTQiIHN0eWxlPSJlbmFibGUtYmFja2dyb3VuZDpuZXcgMCAwIDE0IDE0OyIgeG1sOnNwYWNlPSJwcmVzZXJ2ZSI+CjxzdHlsZSB0eXBlPSJ0ZXh0L2NzcyI+Cgkuc3Qwe2ZpbGw6I0FFQUVBRTt9Cjwvc3R5bGU+CjxwYXRoIGNsYXNzPSJzdDAiIGQ9Ik0xMC45LDUuM0w3LjcsOS4yYy0wLjQsMC40LTEsMC40LTEuNCwwTDMuMSw1LjNDMi45LDUsMy4xLDQuNSwzLjUsNC41aDcuMUMxMC45LDQuNSwxMS4xLDUsMTAuOSw1LjN6Ii8+Cjwvc3ZnPgo=" alt="" class="nav-icon" style="right: 5px; top: 11px; width: 10px;"> is not clickable at point (877, 5). Other element would receive the click: <a data-v-0e4